In [30]:
import pandas as pd
import numpy as np
import os
import json
import statistics
import pandas as pd
import numpy as np
import scipy.stats as stats
import researchpy as rp
import statsmodels.api as sm
from statsmodels.formula.api import ols
    
import matplotlib.pyplot as plt

In [2]:
d = "./result"

dirs_to_analyze = [os.path.join(d, o) for o in os.listdir(d) 
                    if os.path.isdir(os.path.join(d,o)) and "\\experiment" in os.path.join(d, o)]


In [67]:
rowsList = []
for curdir in dirs_to_analyze:
    row = {}
    try:
        config = json.load(open(curdir + "/config.json"))
        row['Executor Memory'] = int(config['jobClassParameters'][0]['systemParameters']['executorMemory'][:1])
        row['Batch Size'] = int(config['jobClassParameters'][0]['hyperParameters']['batchSize'])
        row['Max Epochs'] = int(config['jobClassParameters'][0]['hyperParameters']['maxEpoch'])
        row['Learning Rate'] = float(config['jobClassParameters'][0]['hyperParameters']['learningRate'])
#         row['Learning Rate Decay'] = float(config['jobClassParameters'][0]['hyperParameters']['learningrateDecay'])
        row['Nodes'] = int(config['numberOfNodes'])
        row['Lambda'] = float(config['lambda'])

    except Exception as e:
        print("Error parsing config for folder " + curdir)
        print(e)
        
    try:
        accuracy = pd.read_json(curdir + "/accuracy.json").transpose()
        row['Top 1 mean'] = accuracy["top1"].mean()
        row['Top 1 std'] = accuracy["top1"].std()
        row['Top 5 mean'] = accuracy["top5"].mean()
        row['Top 5 std'] = accuracy["top5"].std()
        row['loss mean'] = accuracy["loss"].mean()
        row['loss std'] = accuracy["loss"].std()
        row['lossnum'] = accuracy["lossnum"].mean()
    except Exception as e:
        print("Error parsing accuracy for folder " + curdir)
        print(e)
        
        
    try:
        time_summary = json.load(open(curdir + "/summary.json"))
        service_times = []
        work_times = []
        process_times = []
        
        for job in time_summary['jobs']:
            service_time = float(job['end']) - float(job['arrive'])
            service_times.append(service_time)
            wait_time = float(job['start']) - float(job['arrive'])
            wait_times.append(wait_time)
            process_time = float(job['end']) - float(job['start'])
            process_times.append(process_time)

        row['Service Time mean'] = statistics.mean(service_times)
        row['Service Time std'] = statistics.stdev(service_times)
        row['Process Time mean'] = statistics.mean(process_times)
        row['Process Time std'] = statistics.stdev(process_times)
        row['Waiting Time mean'] = statistics.mean(wait_times)
        row['Waiting Time std'] = statistics.stdev(wait_times)
    except Exception as e:
        print("Error parsing time summary for folder " + curdir)
        print(e)
        
    
    rowsList.append(row) 
    
analysis = pd.DataFrame(rowsList)
analysis = analysis.drop([25, 21, 10])
analysis

Error parsing accuracy for folder ./result\experiment1_0.01_32_10_2G_2_0.01
Expected object or value
Error parsing time summary for folder ./result\experiment1_0.01_32_10_2G_2_0.01
[Errno 2] No such file or directory: './result\\experiment1_0.01_32_10_2G_2_0.01/summary.json'


,Executor Memory,Batch Size,Max Epochs,Learning Rate,Nodes,Lambda,Top 1 mean,Top 1 std,Top 5 mean,Top 5 std,loss mean,loss std,lossnum,Service Time mean,Service Time std,Process Time mean,Process Time std,Waiting Time mean,Waiting Time std
0,1,128,10,0.001,1,0.01,0.638575,0.040876,0.946263,0.014880,1.907845,0.108069,157.0,1776.609323,989.718032,443.521094,23.668845,948.803662,824.338424
1,1,128,10,0.001,2,0.01,0.606055,0.046856,0.939273,0.019292,1.969239,0.058131,157.0,2120.792949,645.245517,656.160590,330.464160,950.441733,824.938711
2,2,128,10,0.001,1,0.01,0.648443,0.021566,0.957329,0.006531,1.913965,0.066078,157.0,1766.907113,935.187775,475.978486,11.339233,951.196932,825.203317
3,1,128,5,0.001,1,0.01,0.528279,0.065775,0.906986,0.028010,2.105443,0.046931,157.0,1670.351884,860.363370,268.185179,6.783555,953.188597,825.779366
4,1,128,5,0.001,2,0.01,0.573604,0.051022,0.920057,0.024303,2.077709,0.060236,157.0,1321.995722,393.015358,309.784817,80.674195,953.595392,823.757864
5,2,128,5,0.001,1,0.01,0.549371,0.074137,0.915879,0.035446,2.082460,0.049013,157.0,1668.154473,862.796170,268.689102,6.599487,955.542423,824.326468
6,2,128,5,0.001,2,0.01,0.556050,0.040288,0.920758,0.018385,2.100687,0.046055,157.0,1161.089359,375.572796,290.346777,36.715810,954.912331,821.970395
7,1,32,10,0.001,1,0.01,0.724250,0.009970,0.975650,0.000778,1.289145,0.002442,625.0,1954.859985,853.568007,1340.711881,14.968273,954.701462,821.901708
8,1,32,10,0.001,2,0.01,0.733275,0.020606,0.977575,0.000830,1.329912,0.099123,625.0,2075.957269,768.054586,1405.175199,44.450360,954.350511,821.792681
9,2,32,10,0.001,1,0.01,0.718750,0.016476,0.977800,0.001414,1.338503,0.138019,625.0,1959.242099,844.162402,1339.577599,32.175336,954.143787,821.725233


In [82]:
# Process time results:
print("Max avg process time: ", analysis["Process Time mean"].max())
print("Min avg process time: ", analysis["Process Time mean"].min())
print("Mean avg process time: ", analysis["Process Time mean"].mean())

print("Min mu: ", "%.7f" % (1/analysis["Process Time mean"].max()))
print("Max mu: ", "%.7f" % (1/analysis["Process Time mean"].min()))
print("Mean mu: ", "%.7f" % (1/analysis["Process Time mean"].mean()))


Max avg process time:  1603.637364089489
Min avg process time:  266.5028019632612
Mean avg process time:  711.4513971013635
Min mu:  0.0006236
Max mu:  0.0037523
Mean mu:  0.0014056


In [69]:
important_cols=[x for x in analysis.columns if "Top 1" in x]
important_cols.extend([x for x in analysis.columns if "Time" in x])


In [70]:
# Executor Memory
from IPython.display import HTML, display

to_show = analysis.columns[0:5]
for col in to_show:
    display(rp.summary_cont(analysis[important_cols].groupby(analysis[col])))

Top 1 mean                                                    \
                         N      Mean        SD        SE 95% Conf.  Interval   
Executor Memory                                                                
1                       16  0.718374  0.091634  0.022908  0.673474  0.763275   
2                       11  0.703264  0.096758  0.029174  0.646083  0.760444   

                Top 1 std                                ...  \
                        N      Mean        SD        SE  ...   
Executor Memory                                          ...   
1                      16  0.026360  0.016665  0.004166  ...   
2                      11  0.027701  0.020464  0.006170  ...   

                Waiting Time mean                                    \
                               SD        SE   95% Conf.    Interval   
Executor Memory                                                       
1                        3.384475  0.846119  947.812799  951.129584   
2                        3.634512  1.095846  948.080535  952.376253   

                Waiting Time std                                              \
                               N        Mean        SD        SE   95% Conf.   
Executor Memory                                                                
1                             16  823.769970  1.193240  0.298310  823.185282   
2                             11  823.642845  1.113501  0.335733  822.984808   

                             
                   Interval  
Executor Memory              
1                824.354657  
2                824.300882  

[2 rows x 48 columns]

Top 1 mean                                                    \
                    N      Mean        SD        SE 95% Conf.  Interval   
Batch Size                                                                
32                 14  0.754944  0.064873  0.017338  0.720962  0.788926   
128                13  0.666206  0.097156  0.026946  0.613391  0.719020   

           Top 1 std                                ... Waiting Time mean  \
                   N      Mean        SD        SE  ...                SD   
Batch Size                                          ...                     
32                14  0.017903  0.011754  0.003141  ...          3.525291   
128               13  0.036603  0.018736  0.005196  ...          2.208569   

                                             Waiting Time std              \
                  SE   95% Conf.    Interval                N        Mean   
Batch Size                                                                  
32          0.942174  946.197693  949.891014               14  823.054233   
128         0.612547  950.447904  952.849087               13  824.433196   

                                                        
                  SD        SE   95% Conf.    Interval  
Batch Size                                              
32          0.780448  0.208583  822.645409  823.463056  
128         1.045843  0.290065  823.864670  825.001723  

[2 rows x 48 columns]

Top 1 mean                                                    \
                    N      Mean        SD        SE 95% Conf.  Interval   
Max Epochs                                                                
5                  15  0.690280  0.100608  0.025977  0.639365  0.741194   
10                 12  0.739641  0.075745  0.021866  0.696785  0.782498   

           Top 1 std                                ... Waiting Time mean  \
                   N      Mean        SD        SE  ...                SD   
Max Epochs                                          ...                     
5                 15  0.032864  0.019409  0.005011  ...          3.624028   
10                12  0.019460  0.013122  0.003788  ...          3.350644   

                                             Waiting Time std              \
                  SE   95% Conf.    Interval                N        Mean   
Max Epochs                                                                  
5           0.935720  947.853114  951.521137               15  823.818065   
10          0.967248  947.999570  951.791181               12  823.593320   

                                                        
                  SD        SE   95% Conf.    Interval  
Max Epochs                                              
5           0.916258  0.236577  823.354374  824.281755  
10          1.406846  0.406121  822.797322  824.389318  

[2 rows x 48 columns]

Top 1 mean                                                    \
                       N      Mean        SD        SE 95% Conf.  Interval   
Learning Rate                                                                
0.001                 14  0.641098  0.068698  0.018360  0.605112  0.677084   
0.010                 13  0.788810  0.032447  0.008999  0.771171  0.806448   

              Top 1 std                                ... Waiting Time mean  \
                      N      Mean        SD        SE  ...                SD   
Learning Rate                                          ...                     
0.001                14  0.036481  0.019525  0.005218  ...          3.711932   
0.010                13  0.016595  0.007874  0.002184  ...          2.502879   

                                                Waiting Time std              \
                     SE   95% Conf.    Interval                N        Mean   
Learning Rate                                                                  
0.001          0.992056  949.189943  953.078801               14  823.601002   
0.010          0.694174  946.960203  949.681364               13  823.844368   

                                                           
                     SD        SE   95% Conf.    Interval  
Learning Rate                                              
0.001          1.322988  0.353583  822.907978  824.294026  
0.010          0.944652  0.261999  823.330849  824.357887  

[2 rows x 48 columns]

Top 1 mean                                                   Top 1 std  \
               N      Mean        SD        SE 95% Conf.  Interval         N   
Nodes                                                                          
1             16  0.713896  0.091700  0.022925  0.668963  0.758829        16   
2             11  0.709778  0.097373  0.029359  0.652234  0.767322        11   

                                     ... Waiting Time mean            \
           Mean        SD        SE  ...                SD        SE   
Nodes                                ...                               
1      0.027653  0.020574  0.005144  ...          3.580881  0.895220   
2      0.025820  0.014121  0.004258  ...          3.395349  1.023736   

                              Waiting Time std                        \
        95% Conf.    Interval                N        Mean        SD   
Nodes                                                                  
1      948.052490  951.561754               16  823.876087  1.232892   
2      947.733244  951.746291               11  823.488493  1.004088   

                                         
             SE   95% Conf.    Interval  
Nodes                                    
1      0.308223  823.271970  824.480204  
2      0.302744  822.895115  824.081871  

[2 rows x 48 columns]

In [80]:
display(rp.summary_cont(analysis[[x for x in important_cols if "Process" in x]].groupby(analysis["Batch Size"])))

Process Time mean                                                  \
                           N         Mean          SD         SE   95% Conf.   
Batch Size                                                                     
32                        14  1014.841356  348.205774  93.061908  832.440017   
128                       13   384.723749  129.077480  35.799652  314.556432   

                        Process Time std                                   \
               Interval                N       Mean         SD         SE   
Batch Size                                                                  
32          1197.242695               14  52.372110  57.971920  15.493647   
128          454.891066               13  46.978667  89.491434  24.820458   

                                  
            95% Conf.   Interval  
Batch Size                        
32          22.004562  82.739659  
128         -1.669431  95.626764

26.727289401491483